In [1]:
from sklearn import preprocessing as pp
import numpy as np
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from numpy.random import multinomial as randm

# Current Counts = Current Counts + Note Length % 16 ?

def buildNotes():
    
    print("In buildNotes()")
    
    notes = []
    noteLengths = []
    
    for file in glob.glob("Music/*.mid"):
        
        try:
            midi = converter.parse(file)    
            
        except:
            print("MIDI file %s failed to parse" % file)
            continue
            
        print("Parsing %s" % file)
        notesToParse = None
        
        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notesToParse = s2.parts[0].recurse()
            print("Instrument Parts %d" % len(s2.parts))
            
        except: # file has notes in a flat structure
            notesToParse = midi.flat.notes
            
        # For each note in notesToParse (a stream of notes)
            
        for element in notesToParse:
            
            # Interesting parameters of notes that we may want to look at here.
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                noteLengths.append(str(element.quarterLength))
                
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                noteLengths.append(str(element.quarterLength))

    pickle.dump(notes, open('notes.p', 'wb'))
    pickle.dump(noteLengths, open('noteLengths.p', 'wb'))

    return notes, noteLengths

def prepareSeq(notes, noteLengths):

    seqLength = 4
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    # We only have 2 dimensions at the moment, pitch and length
    # noteIn - a collection of categorical arrays, each row is one category
    # noteOut - the integer which converts to the index of that category's respective
    # one hot encoding.  These categorical one hot encodings are later concatenated
    # columnwise to form a multi hot encoding.

    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]

    # Mapping of i:i+seqLength notes to note i + seqLength
    
    # X[0] sequence_in : [seqLength]
    # X[1] length_in : [seqLength]
    
    # Y[0] sequence_out : 1
    # Y[1] length_out : 1
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        sequence_out = notes[i + seqLength]
        
        length_in = noteLengths[i:i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteIn[1].append([lengthToInt[char] for char in length_in])
        
        noteOut[0].append(noteToInt[sequence_out])
        noteOut[1].append(lengthToInt[length_out])
        
    # npatterns = len(noteIn) (inputSize)

    # reshape the input into a format compatible with LSTM layers
    # noteIn = np.reshape(noteIn, (inputSize, seqLength, 1))
    
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
    
    print("noteIn shape:")
    print(noteIn.shape)
    print("noteOut shape:")
    print(noteOut.shape)
    print(noteIn)
    
    catNote = np_utils.to_categorical(noteOut[0])
    catLength = np_utils.to_categorical(noteOut[1])
    
    # Concatenate the input arrays row wise
    # Concatenate the categorical arrays along axis 1 (columnwise)
    
    print("noteIn:")
    print(noteIn)
    
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    dout = np.concatenate((catNote, catLength), 1)
    din = np.reshape(reshape, (inputSize, seqLength, categories))
    
    print("din shape:")
    print(din.shape)
    print("dout shape:")
    print(dout.shape)
    print("din:")
    print(din)
    
    print(len(catNote[0]))
    print(len(catLength[0]))
    
    print(din[0][0][0])
    print(din[0][0][1])

    return (din, dout)


Using TensorFlow backend.
C:\Users\root\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\root\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\root\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\root\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [47]:
# print(np.arange(10).reshape(5,2).T.flatten())
notes, noteLengths = buildNotes()
noteIn, noteOut = prepareSeq(notes, noteLengths)
print(noteIn.shape)
print(noteOut.shape)

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333 0.13333333 0.13333333 0.2       ]
  [0.13333333 0.13333333 0.2        0.        ]
  [0.13333333 0.2        0.         0.        ]
  ...
  [0.06666667 0.8        0.06666667 0.13333333]
  [0.8        0.06666667 0.13333333 0.06666667]
  [0.06666667 0.13333333 0.06666667 0.66666667]]]
noteIn:
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333

In [48]:
def createNet(noteIn, nvocab):

    print("In createNet()")
    
    print(noteIn.shape)
    
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    
    # Chris's code
    model.add(LSTM(
        512,
        input_shape=(noteIn.shape[1], noteIn.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=False
    ))
    
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation(activation='relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    
    #model.add(Flatten())
    
    model.add(Dense(nvocab))
    model.add(Lambda(lambda x: x / 0.6))
    model.add(Activation(activation='softmax'))

    model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
    
    return model

def trainNet(_epochs=1):
    """ Train a Neural Network to generate music """
    notes, noteLengths = buildNotes()

    nvocab = len(set(notes)) + len(set(noteLengths))
    
    noteIn, noteOut = prepareSeq(notes, noteLengths)
    
    model = createNet(noteIn, nvocab)
 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    callbacks_list = [checkpoint]

    # Your line of code here
    model.fit(noteIn, noteOut, batch_size=64, epochs=_epochs, verbose=1, callbacks=callbacks_list)
    


In [ ]:
trainNet(50)

In buildNotes()
Parsing Music\Fugue1.mid
noteIn shape:
(2, 736, 4)
noteOut shape:
(2, 736)
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333 0.13333333 0.13333333 0.2       ]
  [0.13333333 0.13333333 0.2        0.        ]
  [0.13333333 0.2        0.         0.        ]
  ...
  [0.06666667 0.8        0.06666667 0.13333333]
  [0.8        0.06666667 0.13333333 0.06666667]
  [0.06666667 0.13333333 0.06666667 0.66666667]]]
noteIn:
[[[0.46511628 0.55813953 0.65116279 0.81395349]
  [0.55813953 0.65116279 0.81395349 0.95348837]
  [0.65116279 0.81395349 0.95348837 0.81395349]
  ...
  [0.97674419 0.1627907  0.8372093  0.58139535]
  [0.1627907  0.8372093  0.58139535 0.34883721]
  [0.8372093  0.58139535 0.34883721 0.51162791]]

 [[0.13333333

In [71]:
def generate(model):
    
    print("In Generate")
    
    notes = pickle.load(open('notes.p', 'rb'))
    noteLengths = pickle.load(open('noteLengths.p', 'rb'))

    pitchSet = sorted(set(notes))
    lengthSet = sorted(set(noteLengths))
    
    # 1. Call to prepareSeqPred
    
    noteIn = prepareSeqPred(notes, noteLengths) 
    
    # print("Model Loaded")
    
    # 2. Call to genNotes
    
    predOut = genNotes(model, noteIn, pitchSet, lengthSet)
    
    # 3. Call to createMidi
    createMidi(predOut)

# 1.
    
def prepareSeqPred(notes, noteLengths):
    
    print("In Prepare Sequences Prediction")
    
    seqLength = 4
    inputSize = len(notes) - seqLength
    categories = 2
    
    pitchSet = sorted(set(notes))
    npitch = len(pitchSet)
    noteToInt = dict((note, number) for number, note in enumerate(pitchSet))
    
    lengthSet = sorted(set(noteLengths))
    nlengths = len(lengthSet)
    lengthToInt = dict((length, number) for number, length in enumerate(lengthSet))
    
    noteIn = [
        [],
        []
    ]
    
    noteOut = [
        [],
        []
    ]
    
    for i in range(0, inputSize, 1):
        
        sequence_in = notes[i:i + seqLength]
        length_in = noteLengths[i:i + seqLength]
        
        sequence_out = notes[i + seqLength]
        length_out = noteLengths[i + seqLength]
        
        noteIn[0].append([noteToInt[char] for char in sequence_in])
        noteOut[0].append(noteToInt[sequence_out])
        
        noteIn[1].append([lengthToInt[char] for char in length_in])
        noteOut[1].append(lengthToInt[length_out])
        
    noteIn = np.array(noteIn, dtype=float)
    noteOut = np.array(noteOut, dtype=float)
    
    # normalize input
    noteIn[0] = noteIn[0] / float(npitch)
    noteIn[1] = noteIn[1] / float(nlengths)
        
    reshape = []
    for N in range(inputSize):
        for S in range(seqLength):
            for C in range(categories):
                reshape.append(noteIn[C][N][S])
    
    din = np.reshape(reshape, (inputSize, seqLength, categories))
        
    print("din shape:")
    print(din.shape)
    print("din:")
    print(din)

    return din

# 2.

def genNotes(model, noteIn, pitchSet, lengthSet):
    
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    
    inputSize = len(noteIn)
        
    start = np.random.randint(0, inputSize - 1)
    pattern = noteIn[start]
    
    print("pattern %s" % pattern)
        
    seqLength = len(pattern)
    categories = 2
    
    print("seqLength: %d" % seqLength)

    npitches = len(pitchSet)
    nlengths = len(lengthSet)
    
    print("npitches %d nlengths %d" % (npitches, nlengths))
    
    # TODO
    intToNote = dict((number, note) for number, note in enumerate(pitchSet))
    intToLength = dict((number, length) for number, length in enumerate(lengthSet))
    
    predOut = []
    
    print("In genNotes(): noteIn[start] = %s" % pattern)
    print("inputSize: %d" % inputSize)
    print("categories: %d" % categories)

    for i in range(3):
        
        prediction_input = np.reshape(pattern, (1, seqLength, categories))
        
        print("prediction_input:")
        print(prediction_input)
        
        ### Complete the line below
        prediction = model.predict(np.array(prediction_input))
        print(len(prediction[0]))
        
        print("prediction: %s" % prediction[0])
        print("prediction length: %d" % len(prediction[0]))
        
        sumNote = sum(prediction[0][:npitches])
        sumLength = sum(prediction[0][npitches:])
        
        print("sumNote %f sumLength %f" % (sumNote, sumLength))
        
        predPitch = prediction[0][:npitches]
        predLength = prediction[0][npitches:]
        
        print("Before")
        print(predPitch)
        print(predLength)
        
        #predPitch /= sumNote
        #predLength /= sumLength
        
        predPitch /= predPitch.sum()
        predLength /= predLength.sum()
        
        print("After")
        print(predPitch)
        print(predLength)
        
        array = randm(1, predPitch)
        indexPitch = np.argmax(array)
        
        array = randm(1, predLength)
        indexLength = np.argmax(array)
        
        #TODO return tuple pitch, length
        result = np.array([intToNote[indexPitch], intToLength[indexLength]])
        predOut.append(result)
        
        print("result %s" % result)
        
        append = np.reshape(np.array([indexPitch, indexLength]), (1,2))
        pattern = np.concatenate((pattern, append))
        
        print("pattern: %s" % pattern)
        
        pattern = pattern[1:len(pattern)]

    return predOut

# 3.
    
def createMidi(predOut):
    print("In createMidi()")
    offset = 0
    output_notes = []
    for pattern in predOut:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')


In [6]:
model = 0

In [75]:
# Load a previous model (dependency)
if model == 0:
    model = load_model("weights2-improvement-01-10.7344-bigger.hdf5")
generate(model)

In Generate
In Prepare Sequences Prediction
din shape:
(736, 4, 2)
din:
[[[0.46511628 0.13333333]
  [0.55813953 0.13333333]
  [0.65116279 0.13333333]
  [0.81395349 0.2       ]]

 [[0.55813953 0.13333333]
  [0.65116279 0.13333333]
  [0.81395349 0.2       ]
  [0.95348837 0.        ]]

 [[0.65116279 0.13333333]
  [0.81395349 0.2       ]
  [0.95348837 0.        ]
  [0.81395349 0.        ]]

 ...

 [[0.97674419 0.06666667]
  [0.1627907  0.8       ]
  [0.8372093  0.06666667]
  [0.58139535 0.13333333]]

 [[0.1627907  0.8       ]
  [0.8372093  0.06666667]
  [0.58139535 0.13333333]
  [0.34883721 0.06666667]]

 [[0.8372093  0.06666667]
  [0.58139535 0.13333333]
  [0.34883721 0.06666667]
  [0.51162791 0.66666667]]]
pattern [[0.3255814  0.13333333]
 [0.11627907 0.        ]
 [0.93023256 0.13333333]
 [0.65116279 0.06666667]]
seqLength: 4
npitches 43 nlengths 15
In genNotes(): noteIn[start] = [[0.3255814  0.13333333]
 [0.11627907 0.        ]
 [0.93023256 0.13333333]
 [0.65116279 0.06666667]]
inputSiz

AttributeError: 'numpy.ndarray' object has no attribute 'isdigit'

In [ ]:
generate(model)